# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 22 2022 8:30AM,240117,10,6616005,Yusen – 3D Matrix,Released
1,Jun 22 2022 8:30AM,240119,10,8175160,Yusen – 3D Matrix,Released
2,Jun 21 2022 3:45PM,240121,16,SAR00000344.2,Sartorius Stedim North America,Released
3,Jun 21 2022 3:41PM,240120,19,ACG-2102490401,ACG North America LLC,Released
4,Jun 21 2022 3:37PM,240118,19,ADV80003469,"AdvaGen Pharma, Ltd",Released
5,Jun 21 2022 3:37PM,240118,19,ADV80003470,"AdvaGen Pharma, Ltd",Executing
6,Jun 21 2022 3:37PM,240118,19,ADV80003471,"AdvaGen Pharma, Ltd",Released
7,Jun 21 2022 3:37PM,240118,19,ADV80003472,"AdvaGen Pharma, Ltd",Released
8,Jun 21 2022 3:37PM,240118,19,ADV80003473,"AdvaGen Pharma, Ltd",Released
9,Jun 21 2022 3:37PM,240118,19,ADV80003474,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
38,240118,Executing,1
39,240118,Released,21
40,240119,Released,1
41,240120,Released,1
42,240121,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240117,NaN,NaN,1.0
240118,NaN,1.0,21.0
240119,NaN,NaN,1.0
240120,NaN,NaN,1.0
240121,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240003,0.0,0.0,1.0
240007,0.0,0.0,1.0
240030,0.0,0.0,1.0
240034,0.0,0.0,9.0
240037,14.0,4.0,4.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240003,0,0,1
240007,0,0,1
240030,0,0,1
240034,0,0,9
240037,14,4,4


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240003,0,0,1
1,240007,0,0,1
2,240030,0,0,1
3,240034,0,0,9
4,240037,14,4,4


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240003,,,1
1,240007,,,1
2,240030,,,1
3,240034,,,9
4,240037,14,4,4


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 22 2022 8:30AM,240117,10,Yusen – 3D Matrix
1,Jun 22 2022 8:30AM,240119,10,Yusen – 3D Matrix
2,Jun 21 2022 3:45PM,240121,16,Sartorius Stedim North America
3,Jun 21 2022 3:41PM,240120,19,ACG North America LLC
4,Jun 21 2022 3:37PM,240118,19,"AdvaGen Pharma, Ltd"
26,Jun 21 2022 3:29PM,240116,16,Sartorius Lab Products and Service
32,Jun 21 2022 3:25PM,240114,19,"NAPP Technologies, LLC"
33,Jun 21 2022 3:23PM,240113,10,Yusen – 3D Matrix
34,Jun 21 2022 3:18PM,240112,10,ISDIN Corporation
46,Jun 21 2022 3:10PM,240111,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 22 2022 8:30AM,240117,10,Yusen – 3D Matrix,,,1
1,Jun 22 2022 8:30AM,240119,10,Yusen – 3D Matrix,,,1
2,Jun 21 2022 3:45PM,240121,16,Sartorius Stedim North America,,,1
3,Jun 21 2022 3:41PM,240120,19,ACG North America LLC,,,1
4,Jun 21 2022 3:37PM,240118,19,"AdvaGen Pharma, Ltd",,1,21
5,Jun 21 2022 3:29PM,240116,16,Sartorius Lab Products and Service,,,6
6,Jun 21 2022 3:25PM,240114,19,"NAPP Technologies, LLC",,,1
7,Jun 21 2022 3:23PM,240113,10,Yusen – 3D Matrix,,,1
8,Jun 21 2022 3:18PM,240112,10,ISDIN Corporation,,,12
9,Jun 21 2022 3:10PM,240111,10,ISDIN Corporation,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 22 2022 8:30AM,240117,10,Yusen – 3D Matrix,1,,
1,Jun 22 2022 8:30AM,240119,10,Yusen – 3D Matrix,1,,
2,Jun 21 2022 3:45PM,240121,16,Sartorius Stedim North America,1,,
3,Jun 21 2022 3:41PM,240120,19,ACG North America LLC,1,,
4,Jun 21 2022 3:37PM,240118,19,"AdvaGen Pharma, Ltd",21,1,
5,Jun 21 2022 3:29PM,240116,16,Sartorius Lab Products and Service,6,,
6,Jun 21 2022 3:25PM,240114,19,"NAPP Technologies, LLC",1,,
7,Jun 21 2022 3:23PM,240113,10,Yusen – 3D Matrix,1,,
8,Jun 21 2022 3:18PM,240112,10,ISDIN Corporation,12,,
9,Jun 21 2022 3:10PM,240111,10,ISDIN Corporation,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 22 2022 8:30AM,240117,10,Yusen – 3D Matrix,1,,
1,Jun 22 2022 8:30AM,240119,10,Yusen – 3D Matrix,1,,
2,Jun 21 2022 3:45PM,240121,16,Sartorius Stedim North America,1,,
3,Jun 21 2022 3:41PM,240120,19,ACG North America LLC,1,,
4,Jun 21 2022 3:37PM,240118,19,"AdvaGen Pharma, Ltd",21,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 22 2022 8:30AM,240117,10,Yusen – 3D Matrix,1,NaN,NaN
1,Jun 22 2022 8:30AM,240119,10,Yusen – 3D Matrix,1,NaN,NaN
2,Jun 21 2022 3:45PM,240121,16,Sartorius Stedim North America,1,NaN,NaN
3,Jun 21 2022 3:41PM,240120,19,ACG North America LLC,1,NaN,NaN
4,Jun 21 2022 3:37PM,240118,19,"AdvaGen Pharma, Ltd",21,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 22 2022 8:30AM,240117,10,Yusen – 3D Matrix,1,0.0,0.0
1,Jun 22 2022 8:30AM,240119,10,Yusen – 3D Matrix,1,0.0,0.0
2,Jun 21 2022 3:45PM,240121,16,Sartorius Stedim North America,1,0.0,0.0
3,Jun 21 2022 3:41PM,240120,19,ACG North America LLC,1,0.0,0.0
4,Jun 21 2022 3:37PM,240118,19,"AdvaGen Pharma, Ltd",21,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3361186,83,13.0,14.0
102,240089,1,0.0,0.0
15,720177,8,40.0,0.0
16,480237,7,0.0,0.0
18,480215,2,0.0,0.0
19,1680701,28,1.0,0.0
20,960259,32,35.0,0.0
22,240003,1,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3361186,83,13.0,14.0
1,102,240089,1,0.0,0.0
2,15,720177,8,40.0,0.0
3,16,480237,7,0.0,0.0
4,18,480215,2,0.0,0.0
5,19,1680701,28,1.0,0.0
6,20,960259,32,35.0,0.0
7,22,240003,1,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,83,13.0,14.0
1,102,1,0.0,0.0
2,15,8,40.0,0.0
3,16,7,0.0,0.0
4,18,2,0.0,0.0
5,19,28,1.0,0.0
6,20,32,35.0,0.0
7,22,1,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,83.0
1,102,Released,1.0
2,15,Released,8.0
3,16,Released,7.0
4,18,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,15,16,18,19,20,22
Status,,,,,,,,
Completed,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,13.0,0.0,40.0,0.0,0.0,1.0,35.0,0.0
Released,83.0,1.0,8.0,7.0,2.0,28.0,32.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,15,16,18,19,20,22
0,Completed,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,13.0,0.0,40.0,0.0,0.0,1.0,35.0,0.0
2,Released,83.0,1.0,8.0,7.0,2.0,28.0,32.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,15,16,18,19,20,22
0,Completed,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,13.0,0.0,40.0,0.0,0.0,1.0,35.0,0.0
2,Released,83.0,1.0,8.0,7.0,2.0,28.0,32.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()